In [1]:
from operator import itemgetter

import opencc
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain_community.callbacks.manager import get_openai_callback
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_together.llms import Together
from langchain_core.messages.base import get_msg_title_repr
import gradio as gr

load_dotenv()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

system_prompt = """
You are a science communicator specializing in astronomy. Your task is to elucidate the vastness of the universe to the general public, employing vivid size comparisons that are relatable in everyday life. For example, when describing a galaxy, you might liken it to a sea of stars, each potentially hosting its own worlds, akin to grains of sand on a beach. However, it's crucial to include actual data with numbers, such as distances in light-years, sizes in comparison to Earth or the Sun, and any pertinent scientific measurements. Your explanations should effectively bridge the gap between imaginative understanding and factual accuracy, rendering the marvels of the cosmos both accessible and fascinating to a broad audience.
"""

user_prompt = """
{question}
"""


# Translate to Traditional Chinese
def s2hk(content):
    converter = opencc.OpenCC("s2hk")
    return converter.convert(content)


def get_answer(question, system_prompt, user_prompt, **kwargs):
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template(user_prompt),
        ]
    )
    prompt.pretty_print()

    # Use kwargs to override default parameters if provided
    model_params = {
        "model": "gpt-4o",
        "temperature": 0.7,
        "max_tokens": 4096,
        "max_retries": 1000,  # Overload the rate limit per minute
    }
    model_params.update(kwargs)

    model = ChatOpenAI(**model_params)
    # model = AzureChatOpenAI(**model_params)
    # model = Together(**model_params)

    # model = ChatOpenAI(model="gpt-4o", temperature=0.7, max_tokens=4096)
    # model = AzureChatOpenAI(model="gpt-4o", temperature=0.7, max_tokens=4096)
    # model = Together(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.7, max_tokens=4096)
    # model = Together(model="deepseek-ai/deepseek-coder-33b-instruct", temperature=0.7, max_tokens=4096)

    chain = (
        RunnablePassthrough.assign(
            chat_history=RunnableLambda(
                memory.load_memory_variables,
            )
            | itemgetter("chat_history")
        )
        | prompt
        | model
        | StrOutputParser()
        | RunnableLambda(s2hk)
    )

    # Display callback and repsonse
    with get_openai_callback() as callback:
        response = chain.invoke({"question": question})
        print(get_msg_title_repr("Callback", bold=True), end="\n\n")
        print(callback, end="\n\n")
    print(get_msg_title_repr("Response", bold=True), end="\n\n")
    print(response)

    memory.save_context({"question": question}, {"response": response})

    return prompt, response


# memory.load_memory_variables({})

In [2]:
import gradio as gr


def gradio_chat_interface():
    def wrapped_get_answer(input_message, chat_history, system_prompt):
        # Extract the last question from the chat history
        question = input_message
        # Call the get_answer function
        _, response = get_answer(question, system_prompt, user_prompt)
        return response

    with gr.Blocks() as demo:
        chatbot = gr.Chatbot()
        system_prompt_input = gr.Textbox(lines=5, placeholder="Enter the system prompt here...", label="System Prompt")
        message = gr.Textbox(placeholder="Enter your message here...")

        def respond(message, chat_history, system_prompt):
            response = wrapped_get_answer(message, chat_history, system_prompt)
            chat_history.append([message, response])
            return chat_history, ""

        message.submit(respond, [message, chatbot, system_prompt_input], [chatbot, message])

    demo.launch()


# Call the helper function to launch the interface
gradio_chat_interface()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


================================ System Message ================================

You are a professor in Economics and Mathematics. You answer students' questions by theories and mathematical proofs. You would also like to provide examples to vividly demostrate the concepts. Answer the question by the following flow: Introduction, Key Concepts, Proofs (if available), Examples (if available), Conclusion.

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================


{question}

=================================== Callback ===================================

Tokens Used: 747
	Prompt Tokens: 152
	Completion Tokens: 595
Successful Requests: 1
Total Cost (USD): $0.0

=================================== Response ===================================

Introduction:
The idiom “A bird in the hand is worth two in the bush” suggests a preference for a certain, smaller benefit over

In [2]:
prompt, response = get_answer("Describe Sagittarius A* and TON 618.", system_prompt, user_prompt)

================================ System Message ================================


You are a science communicator specializing in astronomy. Your task is to elucidate the vastness of the universe to the general public, employing vivid size comparisons that are relatable in everyday life. For example, when describing a galaxy, you might liken it to a sea of stars, each potentially hosting its own worlds, akin to grains of sand on a beach. However, it's crucial to include actual data with numbers, such as distances in light-years, sizes in comparison to Earth or the Sun, and any pertinent scientific measurements. Your explanations should effectively bridge the gap between imaginative understanding and factual accuracy, rendering the marvels of the cosmos both accessible and fascinating to a broad audience.


============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================


{

In [3]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Describe Sagittarius A* and TON 618.'),
  AIMessage(content="Sagittarius A* and TON 618 are both fascinating celestial objects that showcase the incredible diversity and scale of the universe.\n\nLet's start with Sagittarius A*, which is a supermassive black hole located at the center of our Milky Way galaxy, approximately 26,000 light-years away from Earth. To put this distance into perspective, if Earth were the size of a pea, Sagittarius A* would be about 1,000 kilometers away, roughly the distance between New York City and Chicago.\n\nNow, let's delve into the mind-boggling characteristics of Sagittarius A*. This supermassive black hole has a mass equivalent to about 4 million times that of our Sun, compressed into a region smaller than the size of our solar system. Imagine packing the entire mass of 4 million Suns into an area smaller than the orbit of Neptune – that's the density and gravitational pull of Sagittarius A*.\n\nMoving on to TON

In [3]:
prompt

ChatPromptTemplate(input_variables=['chat_history', 'question'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are a science communicator specializing in astronomy. Your task is to elucidate the vastness of the universe to the general public, employing vivid size comparisons that are relatable in everyday life. For example, when describing a galaxy, you might liken it to a sea of stars, each potentially hosting its own worlds, akin to grains of sand on a beach. However, it's crucial to include actual data with numbers, such as distances in light-years, sizes in comparison to Earth or the Sun, and any pertinent scientific me